In [ ]:
import requests
import numpy as np
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt  # For plotting
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D

def fetch_csv_files(user, repo, folder):

    url = f"https://api.github.com/repos/{user}/{repo}/contents/{folder}"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error fetching folder '{folder}': {response.status_code}")

    files = sorted(response.json(), key=lambda x: x['name'])

    channel_data_list = []
    channel_names = []

    for file_info in files:
        if file_info['name'].endswith('.csv'):
            download_url = file_info['download_url']
            file_response = requests.get(download_url)
            if file_response.status_code != 200:
                raise Exception(f"Error downloading file {file_info['name']}")

            # Use header=0 to indicate the first row is the header
            df = pd.read_csv(StringIO(file_response.text), header=0)
            channel_data = df.values.flatten()
            channel_data_list.append(channel_data)
            channel_names.append(file_info['name'])

    return channel_data_list, channel_names

def sliding_window(data, window_size, step):

    windows = []
    for start in range(0, len(data) - window_size + 1, step):
        window = data[start:start+window_size]
        windows.append(window)
    return np.array(windows)

def load_and_preprocess(user, repo, N, K):

    folders = ['L5', 'LEG', 'ML']

    channels = []
    channel_names = []

    for folder in folders:
        ch_data, ch_names = fetch_csv_files(user, repo, folder)
        channels.extend(ch_data)
        channel_names.extend(ch_names)

    if len(channels) == 0:
        raise ValueError("No CSV files were found in the specified repository folders.")

    # Find shortest channel length
    lengths = [len(ch) for ch in channels]
    T = min(lengths)
    print(f"Minimum channel length across all channels: {T} samples")

    if T < N:
        raise ValueError(f"Not enough samples in channels. Required window size is {N}, "
                         f"but the shortest channel has only {T} samples.")

    truncated_channels = [ch[:T] for ch in channels]

    # Compute step size from overlap ratio
    step = int(N * (1 - K))
    if step <= 0:
        raise ValueError("Step size computed from N and K must be positive.")

    # Compute number of windows (W) after truncation
    W = (T - N) // step + 1
    print(f"Extracting {W} windows of length {N} each (with step size {step}).")

    # Process each truncated channel: extract sliding windows
    processed_channels = []
    for idx, ch in enumerate(truncated_channels):
        windows = sliding_window(ch, N, step)
        if windows.shape[0] != W:
            raise ValueError(f"Channel {idx} ({channel_names[idx]}) produced {windows.shape[0]} windows, but expected {W}.")
        processed_channels.append(windows)
        print(f"Channel [{idx}]: {channel_names[idx]}")  # Print channel index & name

    processed_data = np.stack(processed_channels, axis=0)
    print(f"Processed data shape: {processed_data.shape}")

    return processed_data, channel_names

#def add_synthetic_channel(data, ch1_idx, ch2_idx, channel_names):

#    num_channels, num_windows, num_samples = data.shape
#
#    if ch1_idx >= num_channels or ch2_idx >= num_channels:
#        raise ValueError(f"Channel indices out of range. Data has {num_channels} channels.")
#
#    print(f"Computing synthetic channel using absolute differences between channels {ch1_idx} ({channel_names[ch1_idx]}) and {ch2_idx} ({channel_names[ch2_idx]}).")
#
#    # Compute absolute difference between selected channels
#    synthetic_channel = np.abs(data[ch1_idx].astype(float) - data[ch2_idx].astype(float))
#
#    # Append to the existing dataset
#    updated_data = np.vstack([data, np.expand_dims(synthetic_channel, axis=0)])
#    channel_names.append(f"Synthetic_{channel_names[ch1_idx]}_{channel_names[ch2_idx]}")
#
#    print(f"New data shape after adding synthetic channel: {updated_data.shape}")
#    return updated_data, channel_names

def add_synthetic_channel(data, ch1_idx, ch2_idx, channel_names, dt):

    num_channels, num_windows, num_samples = data.shape

    if ch1_idx >= num_channels or ch2_idx >= num_channels:
        raise ValueError(f"Channel indices out of range. Data has {num_channels} channels.")

    print(f"Computing synthetic channel (rotation difference) using channels {ch1_idx} ({channel_names[ch1_idx]}) and {ch2_idx} ({channel_names[ch2_idx]}).")

    integrated_ch1 = np.cumsum(data[ch1_idx].astype(float) * dt, axis=-1)
    integrated_ch2 = np.cumsum(data[ch2_idx].astype(float) * dt, axis=-1)

    synthetic_channel = np.abs(integrated_ch1 + integrated_ch2)

    # Append the synthetic channel to the existing dataset.
    updated_data = np.vstack([data, np.expand_dims(synthetic_channel, axis=0)])
    channel_names.append(f"Synthetic_RotationDiff_{channel_names[ch1_idx]}_{channel_names[ch2_idx]}")

    print(f"New data shape after adding synthetic rotation difference channel: {updated_data.shape}")
    return updated_data, channel_names


def create_autoencoder_model(input_shape):

    input_layer = Input(shape=input_shape)

    # Encoder
    x = Conv1D(32, kernel_size=3, activation="relu", padding="same")(input_layer)
    x = MaxPooling1D(pool_size=2, padding="same")(x)
    x = Conv1D(16, kernel_size=3, activation="relu", padding="same")(x)
    x = MaxPooling1D(pool_size=2, padding="same")(x)

    # Bottleneck
    x = Conv1D(8, kernel_size=3, activation="relu", padding="same")(x)

    # Decoder
    x = UpSampling1D(size=2)(x)
    x = Conv1D(16, kernel_size=3, activation='relu', padding="same")(x)
    x = UpSampling1D(size=2)(x)
    x = Conv1D(32, kernel_size=3, activation='relu', padding="same")(x)

    output_layer = Conv1D(input_shape[-1], kernel_size=3, activation="sigmoid", padding="same")(x)

    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.summary()

    return autoencoder

def permutation_importance_autoencoder(model, X, channel_names):

    reconstructed_data = model.predict(X)
    baseline_errors = np.mean(np.square(X - reconstructed_data), axis=(1, 2))
    baseline_score = np.mean(baseline_errors)

    importances = []
    num_channels = X.shape[-1]

    for i in range(num_channels):
        X_permuted = X.copy()
        # Permute the data for channel i
        X_permuted[:, :, i] = np.random.permutation(X_permuted[:, :, i])
        reconstructed_permuted = model.predict(X_permuted)
        permuted_errors = np.mean(np.square(X_permuted - reconstructed_permuted), axis=(1, 2))
        permuted_score = np.mean(permuted_errors)

        importance = permuted_score - baseline_score
        importances.append(importance)
        print(f"Channel {i} ({channel_names[i]}) importance: {importance:.4f}")

    return importances

def plot_importance_scores(importances, channel_names):

    plt.figure(figsize=(10, 6))
    plt.bar(channel_names, importances, color='skyblue')
    plt.xlabel("Channels")
    plt.ylabel("Increase in MSE (Permutation Importance)")
    plt.title("Channel Permutation Importances")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

def normalize_data(data):

    # Make a copy to avoid modifying the original data
    normalized_data = data.copy()
    num_channels = normalized_data.shape[-1]

    for i in range(num_channels):
        channel_data = normalized_data[:, :, i]
        min_val = np.min(channel_data)
        max_val = np.max(channel_data)
        if max_val - min_val > 0:
            normalized_data[:, :, i] = (channel_data - min_val) / (max_val - min_val)
        else:
            normalized_data[:, :, i] = 0.0  # If no variation, set to 0

    return normalized_data

def compute_input_distance(model, X_train, X_input):

    # Compute reconstruction error on the training data
    train_recon = model.predict(X_train)
    train_error = np.mean(np.square(X_train - train_recon))

    # Compute reconstruction error on the input data
    input_recon = model.predict(X_input)
    input_error = np.mean(np.square(X_input - input_recon))

    print("\n--- Input Distance Analysis ---")
    print(f"Average reconstruction error on training data: {train_error:.4f}")
    print(f"Average reconstruction error on input data:    {input_error:.4f}")

    if train_error > 0:
        # Compute relative distance: (difference / training error)
        distance_score = (input_error - train_error) / train_error
        print(f"Input distance score (relative to training data): {distance_score:.4f}")
    else:
        distance_score = np.nan
        print("Training error is zero; cannot compute a relative distance score.")

    return distance_score

def choseChannelToCheck(model, X, channels_to_permute, channel_names):

    # Compute baseline reconstruction error
    baseline_recon = model.predict(X)
    baseline_error = np.mean(np.square(X - baseline_recon))

    # Permute specified channels together
    X_permuted = X.copy()
    for idx in channels_to_permute:
        X_permuted[:, :, idx] = np.random.permutation(X_permuted[:, :, idx])

    # Compute reconstruction error on the permuted data
    permuted_recon = model.predict(X_permuted)
    permuted_error = np.mean(np.square(X_permuted - permuted_recon))

    error_difference = permuted_error - baseline_error
    print(f"\nCombined Permutation of Channels {channels_to_permute} ({[channel_names[i] for i in channels_to_permute]})")
    print(f"Baseline Reconstruction Error: {baseline_error:.4f}")
    print(f"Permuted Reconstruction Error: {permuted_error:.4f}")
    print(f"Error Increase: {error_difference:.4f}")

    # Plot the comparison
    labels = ['Baseline', 'Permuted']
    errors = [baseline_error, permuted_error]
    plt.figure(figsize=(6,4))
    plt.bar(labels, errors, color=['skyblue', 'salmon'])
    plt.ylabel("Reconstruction Error (MSE)")
    plt.title(f"Combined Permutation Effect for Channels: {channels_to_permute}")
    plt.tight_layout()
    plt.show()

    return baseline_error, permuted_error, error_difference

# -------------------------
if __name__ == "__main__":
    github_user = "usernameLoophole"
    repo_name_healthy = "autoencoderDatasetHealthy"
    repo_name_injury = "autoencoderDatasetInjury"
    N = 200
    K = 0.5

    # Load and preprocess healthy data
    dataH, channel_names_H = load_and_preprocess(github_user, repo_name_healthy, N, K)
    # Adjust the indices for the synthetic channel as needed:
    trainData, channel_names_H = add_synthetic_channel(dataH, 11, 17, channel_names_H, 0.01)

    # Load and preprocess injury data (or any new input data)
    dataI, channel_names_I = load_and_preprocess(github_user, repo_name_injury, N, K)
    testData, channel_names_I = add_synthetic_channel(dataI, 11, 17, channel_names_I, 0.01)

    # Print final channel names
    print("\nFinal Healthy Dataset Channels:", channel_names_H)
    print("Final Injury Dataset Channels:", channel_names_I)

    # Train the autoencoder
    num_channels = testData.shape[0]
    autoencoder = create_autoencoder_model(input_shape=(N, num_channels))

    # Reshape trainData and testData to match the expected input shape
    # (num_windows, num_samples, num_channels)
    trainData = trainData.transpose(1, 2, 0)
    testData = testData.transpose(1, 2, 0)
    print("trainData shape:", trainData.shape)
    print("testData shape:", testData.shape)

    # Normalize data channel-wise
    trainData = normalize_data(trainData)
    testData = normalize_data(testData)

    autoencoder.fit(trainData, trainData, epochs=20, batch_size=32, validation_data=(testData, testData))

    # Compute and report how "distant" the input is compared to training data
    distance_score = compute_input_distance(autoencoder, trainData, testData)

    # Compute feature importance for each channel using the test dataset
    importance_scores = permutation_importance_autoencoder(autoencoder, testData, channel_names_I)
    print("Feature Importances:", importance_scores)

    # Plot the permutation importance scores
    plot_importance_scores(importance_scores, channel_names_I)

    # Now, use choseChannelToCheck to evaluate the combined effect of permuting two channels together.
    combined_channels = [9, 15]
    choseChannelToCheck(autoencoder, testData, combined_channels, channel_names_I)


Minimum channel length across all channels: 126281 samples
Extracting 1261 windows of length 200 each (with step size 100).
Channel [0]: L5_accx.csv
Channel [1]: L5_accy.csv
Channel [2]: L5_accz.csv
Channel [3]: L5_gyry.csv
Channel [4]: L5_gyrz.csv
Channel [5]: l5_gyrx.csv
Channel [6]: RT_accx.csv
Channel [7]: RT_accy.csv
Channel [8]: RT_accz.csv
Channel [9]: RT_gyrx.csv
Channel [10]: RT_gyry.csv
Channel [11]: RT_gyrz.csv
Channel [12]: ML_accx.csv
Channel [13]: ML_accy.csv
Channel [14]: ML_accz.csv
Channel [15]: ML_gyrx.csv
Channel [16]: ML_gyry.csv
Channel [17]: ML_gyrz.csv
Processed data shape: (18, 1261, 200)
Computing synthetic channel (rotation difference) using channels 11 (RT_gyrz.csv) and 17 (ML_gyrz.csv).
New data shape after adding synthetic rotation difference channel: (19, 1261, 200)
Minimum channel length across all channels: 35397 samples
Extracting 352 windows of length 200 each (with step size 100).
Channel [0]: L5_accx.csv
Channel [1]: L5_accy.csv
Channel [2]: L5_accz.

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 200, 19)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_6 (Conv1D)                    │ (None, 200, 32)             │           1,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 100, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_7 (Conv1D)                    │ (None, 100, 16)             │           1,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 50, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_8 (Conv1D)                    │ (None, 50, 8)               │             392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling1d_2 (UpSampling1D)       │ (None, 100, 8)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_9 (Conv1D)                    │ (None, 100, 16)             │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling1d_3 (UpSampling1D)       │ (None, 200, 16)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_10 (Conv1D)                   │ (None, 200, 32)             │           1,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 200, 19)             │           1,843 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,611 (29.73 KB)

 Trainable params: 7,611 (29.73 KB)

 Non-trainable params: 0 (0.00 B)

trainData shape: (1261, 200, 19)
testData shape: (352, 200, 19)
Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.0287 - val_loss: 0.0199
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0181 - val_loss: 0.0189
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0143 - val_loss: 0.0144
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0101 - val_loss: 0.0114
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0077 - val_loss: 0.0101
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0067 - val_loss: 0.0100
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0061 - val_loss: 0.0100
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0057 - val_loss: 0.0097
Epoch 9/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0053 - val_loss: 0.0098
Epoch 10/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0050 - val_loss: 0.0099
Epoch 11/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0048 - val_loss: 0.0097